In [17]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [18]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [22]:
df_path = './example_data/SW_TaxaFunc.tsv'
meta_path = './example_data/SW_Meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

original df shape: (3999, 229)
after remove all zero row: (3999, 229)


In [24]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V2', 'V5', 'V1', 'V4', 'V3'}


In [33]:
sw.meta_df

,Sample,Person,Treatment,Sweetener,Sweetener_detail
0,V1_01,V1,PBS,PBS,PBSB
1,V1_02,V1,Treatment,ISO,ISO
2,V1_03,V1,Treatment,SOR,SORC
3,V1_04,V1,Treatment,ERY,ERY
4,V1_05,V1,Treatment,HSH,HSH
...,...,...,...,...,...
185,V5_34,V5,Treatment,ACE2,ACE2
186,V5_35,V5,Treatment,ASP,ASP
187,V5_36,V5,Treatment,NEO006,NEO006
188,V5_37,V5,PBS,PBS,PBSC
